In [ ]:
from keras.models import Model
from keras.layers import Dense, Input,concatenate, Flatten, Embedding
import numpy as np
import ncf_helper as helper
from keras.optimizers import Adagrad, Adam
from keras.regularizers import l2
from keras import initializers

num_predictive_factors = 8

#def init_normal( name=None):
#    return initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None)
#initializations.normal(shape, scale=0.01, name=name)

# embedding size is 2 * num_predictive_factors if MLP is 3 layered

interaction_mx = np.load('input/int_mat.npy')
# load data
inputs, labels = helper.training_data_generation('input/training_data.npy',interaction_mx, 5)
#https://datascience.stackexchange.com/questions/13428/what-is-the-significance-of-model-merging-in-keras
user_input = Input(shape=(1,),name='user_input')
#item_input = Input(shape=(len(one_hot_movies),),name='item_input')
item_input = Input(shape=(1,),name='item_input')
user_embed = Flatten()(Embedding(interaction_mx.shape[0] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name = 'MLP_user_embed', 
                                 embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None))(user_input))
item_embed = Flatten()(Embedding(interaction_mx.shape[1] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name = 'MLP_item_embed',
                                 embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None))(item_input))
merged_embed = concatenate([user_embed, item_embed], axis=1)
mlp_1 = Dense(32, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_1')(merged_embed)
mlp_2 = Dense(16, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_2')(mlp_1)
mlp_3 = Dense(8, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_3')(mlp_2)
main_output = Dense(1,
                    #W_regularizer = l2(0.01),
                    activation='sigmoid',init='lecun_uniform',name='main_output')(mlp_3)

model = Model(inputs=[user_input, item_input], output=main_output)
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(inputs, labels, batch_size = 256, epochs = 10)

#Save weights for full_model
#Save weights for full_model
wmlp1= model.get_layer('mlp_1').get_weights()
np.save('MLP_WE/mlp_1_weights_array0', wmlp1[0])
np.save('MLP_WE/mlp_1_weights_array1', wmlp1[1])
np.save('MLP_WE/mlp_2_weights',model.get_layer('mlp_2').get_weights());
np.save('MLP_WE/mlp_3_weights',model.get_layer('mlp_3').get_weights());
np.save('MLP_WE/mlp_user_embed_weights',model.get_layer('MLP_user_embed').get_weights());
np.save('MLP_WE/mlp_item_embed_weights',model.get_layer('MLP_item_embed').get_weights());

/opt/cloudera/parcels/Anaconda/envs/jupyterHub/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", name="main_output", kernel_initializer="lecun_uniform")`
/opt/cloudera/parcels/Anaconda/envs/jupyterHub/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ma...)`


Epoch 1/10
5965014/5965014 [==============================] - 93s - loss: 0.2610 - acc: 0.8863    
Epoch 2/10
5965014/5965014 [==============================] - 93s - loss: 0.2302 - acc: 0.9000    
Epoch 3/10
5965014/5965014 [==============================] - 93s - loss: 0.2146 - acc: 0.9078    
Epoch 4/10
5965014/5965014 [==============================] - 93s - loss: 0.2061 - acc: 0.9118    
Epoch 5/10
5965014/5965014 [==============================] - 92s - loss: 0.2012 - acc: 0.9142    
Epoch 6/10
5965014/5965014 [==============================] - 93s - loss: 0.1976 - acc: 0.9160    
Epoch 7/10
5965014/5965014 [==============================] - 92s - loss: 0.1948 - acc: 0.9173    
Epoch 8/10
5965014/5965014 [==============================] - 92s - loss: 0.1924 - acc: 0.9185    
Epoch 9/10
5965014/5965014 [==============================] - 92s - loss: 0.1905 - acc: 0.9195    
Epoch 10/10
5965014/5965014 [==============================] - 92s - loss: 0.1889 - acc: 0.9202    


In [ ]:
from keras.models import Model
from keras.layers import Dense, Activation,Embedding,Input,concatenate,multiply,Flatten
import numpy as np
import ncf_helper as helper
import keras.layers as layers
from keras import initializers


num_predictive_factors = 8
batch_size = 2
# embedding size is 2 * num_predictive_factors if MLP is 3 layered

# load data
interaction_mx = np.load('input/int_mat.npy')
inputs, labels = helper.training_data_generation('input/training_data.npy',interaction_mx, 5)

#https://datascience.stackexchange.com/questions/13428/what-is-the-significance-of-model-merging-in-keras
user_input = Input(shape=(1,),name='user_input')
item_input = Input(shape=(1,),name='item_input')

user_embed = Flatten()(Embedding(interaction_mx.shape[0] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name = "user_embed",
                                 embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None))(user_input))
item_embed = Flatten()(Embedding(interaction_mx.shape[1] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name = "item_embed",
                                 embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None))(item_input))
merged_embed = multiply([user_embed, item_embed])

main_output = Dense(1, activation='sigmoid',name='main_output')(merged_embed)

model = Model(inputs=[user_input, item_input], output=main_output)
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(inputs, labels, batch_size = 256, epochs = 10)

user_embed_weights = model.get_layer('user_embed').get_weights()
item_embed_weights = model.get_layer('item_embed').get_weights()
main_output_weights = model.get_layer('main_output').get_weights()

np.save('GMF_WE/GMF_user_embed.npy', user_embed_weights)
np.save('GMF_WE/GMF_item_embed.npy', item_embed_weights)
np.save('GMF_WE/GMF_output_layer.npy', main_output_weights)

/opt/cloudera/parcels/Anaconda/envs/jupyterHub/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ma...)`


Epoch 1/10
5965014/5965014 [==============================] - 68s - loss: 0.2696 - acc: 0.8850    
Epoch 2/10
5965014/5965014 [==============================] - 68s - loss: 0.2174 - acc: 0.9071    
Epoch 3/10
5965014/5965014 [==============================] - 68s - loss: 0.2029 - acc: 0.9140    
Epoch 4/10
5965014/5965014 [==============================] - 68s - loss: 0.1936 - acc: 0.9184    
Epoch 5/10
5965014/5965014 [==============================] - 67s - loss: 0.1828 - acc: 0.9233    
Epoch 7/10
5965014/5965014 [==============================] - 68s - loss: 0.1800 - acc: 0.9247    
Epoch 8/10
5965014/5965014 [==============================] - 68s - loss: 0.1780 - acc: 0.9256    
Epoch 9/10
5965014/5965014 [==============================] - 68s - loss: 0.1765 - acc: 0.9262    
Epoch 10/10
5965014/5965014 [==============================] - 68s - loss: 0.1754 - acc: 0.9267    


In [ ]:

# coding: utf-8

# In[6]:

from keras.models import Model
from keras.layers import Dense, Activation,Embedding,Input,concatenate, multiply,Flatten
import numpy as np
import keras.layers as layers
import ncf_helper as helper
from keras.optimizers import Adagrad, Adam
from keras.regularizers import l2
from keras import initializers

def load_weights(model):
    model.get_layer('MLP_user_embed').set_weights(np.load('MLP_WE/mlp_user_embed_weights.npy'))
    model.get_layer('MLP_item_embed').set_weights(np.load('MLP_WE/mlp_item_embed_weights.npy'))
    
    mlp1_0 = np.load('MLP_WE/mlp_1_weights_array0.npy')
    mlp1_1 = np.load('MLP_WE/mlp_1_weights_array1.npy')    
    model.get_layer('mlp_1').set_weights([mlp1_0,mlp1_1])
    
    model.get_layer('mlp_2').set_weights(np.load('MLP_WE/mlp_2_weights.npy'))
    model.get_layer('mlp_3').set_weights(np.load('MLP_WE/mlp_3_weights.npy'))

    model.get_layer('GMF_user_embed').set_weights(np.load('GMF_WE/GMF_user_embed.npy'))
    model.get_layer('GMF_item_embed').set_weights(np.load('GMF_WE/GMF_item_embed.npy'))
    #model.get_layer('GMF_main_output').set_weights(np.load('GMF_WE/GMF_output_layer.npy'))
    
    return model


num_predictive_factors = 8
batch_size = 256


#----- MLP Model -----
interaction_mx = np.load('input/int_mat.npy')
# load data
inputs, labels = helper.training_data_generation('input/training_data.npy',interaction_mx, 5)
#https://datascience.stackexchange.com/questions/13428/what-is-the-significance-of-model-merging-in-keras
user_input = Input(shape=(1,),name='user_input')
#item_input = Input(shape=(len(one_hot_movies),),name='item_input')
item_input = Input(shape=(1,),name='item_input')
MLP_user_embed = Flatten()(Embedding(interaction_mx.shape[0] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name='MLP_user_embed'
                                 #embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None)
                                    )(user_input))
MLP_item_embed = Flatten()(Embedding(interaction_mx.shape[1] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name='MLP_item_embed'
                                 #embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None)
                                    )(item_input))
MLP_merged_embed = concatenate([MLP_user_embed, MLP_item_embed], axis=1)
mlp_1 = Dense(32, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_1')(MLP_merged_embed)
mlp_2 = Dense(16, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_2')(mlp_1)
mlp_3 = Dense(8, activation='relu',
              #W_regularizer = l2(0.01),
              name='mlp_3')(mlp_2)

#MLP_main_output = Dense(1, activation='sigmoid',name='MLP_main_output')(mlp_3)

#----- GMF Model -----

GMF_user_embed = Flatten()(Embedding(interaction_mx.shape[0] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name='GMF_user_embed'
                                 #embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None)
                                    )(user_input))
GMF_item_embed = Flatten()(Embedding(interaction_mx.shape[1] + 1,
                                 num_predictive_factors * 2,
                                 #W_regularizer = l2(0.01),
                                 input_length=1,
                                 #dropout = 0.3,
                                 name='GMF_item_embed'
                                 #embeddings_initializer = initializers.RandomNormal(mean = 0.0, stddev=0.01, seed=None)
                                    )(item_input))
GMF_merged_embed = multiply([GMF_user_embed, GMF_item_embed])

#GMF_main_output = Dense(1, activation='sigmoid',name='GMF_main_output')(GMF_merged_embed)

#Concatenate with GMF last layer
#MLP_input = Input(shape=(len(one_hot_users),),name='MLP_input') #This may be necessary
gmf_mlp_concatenated = concatenate([mlp_3, GMF_merged_embed], axis=1);


#Feed previous concatenate to NeuMF Layer
NeuMF = Dense(16, activation='sigmoid', name='NeuMF')(gmf_mlp_concatenated)
NeuMF_main_output = Dense(1, activation='sigmoid',name='NeuMF_main_output')(NeuMF)

model = Model(inputs=[user_input, item_input], output=NeuMF_main_output)

model = load_weights(model)

model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(inputs, labels, batch_size = 256, epochs = 10)


/opt/cloudera/parcels/Anaconda/envs/jupyterHub/lib/python3.6/site-packages/ipykernel_launcher.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("Ne...)`


Epoch 1/10
5965014/5965014 [==============================] - 83s - loss: 0.1901 - acc: 0.9207    
Epoch 2/10
5965014/5965014 [==============================] - 82s - loss: 0.1796 - acc: 0.9247    
Epoch 3/10
5965014/5965014 [==============================] - 82s - loss: 0.1757 - acc: 0.9262    
Epoch 4/10
5965014/5965014 [==============================] - 82s - loss: 0.1734 - acc: 0.9271    
Epoch 5/10
5965014/5965014 [==============================] - 82s - loss: 0.1722 - acc: 0.9275    
Epoch 6/10
5965014/5965014 [==============================] - 82s - loss: 0.1715 - acc: 0.9278    
Epoch 7/10
5965014/5965014 [==============================] - 82s - loss: 0.1711 - acc: 0.9280    
Epoch 8/10
5965014/5965014 [==============================] - 82s - loss: 0.1709 - acc: 0.9281    
Epoch 9/10
5965014/5965014 [==============================] - 82s - loss: 0.1707 - acc: 0.9281    
Epoch 10/10
5965014/5965014 [==============================] - 82s - loss: 0.1706 - acc: 0.9281    


In [20]:
test = helper.evaluate_integer_input('input/testing_data.npy', model, 'hit_rate', 'input/int_mat.npy')
print('accuracy rate of: ' + str(test))

accuracy rate of: 0.6317880794701987


In [21]:
model.save('full_model_first_run.h5')

In [19]:
print('hello world')

hello world
